# Relationales Schema
Die langfristige Speicherung in einem Data Warehouse wird häufig mit einem relationalen Datenbanksystem ermöglicht. Dazu existieren zwei einfache Schemata.

In [ ]:
%CREATE :memory: OF vgsales.sql

## Inhaltsverzeichnis
- [Daten](#Daten)
- [Snowflake Schema](#Snowflake-Schema)
- [Star Schema](#Star-Schema)
- [Zusammenfassung](#Zusammenfassung)

## Daten
Wir betrachten erneut den Datensatz der Videospielverkäufe pro Jahrzehnt. Im Gegensatz zum vorangegangenen Abschnitt sind noch einige Attribute hinzugekommen.

In [ ]:
%SCHEMA ONLY sales
FROM sales

Daten ohne Normalisierung zu speichern erzeugt aber die üblichen Probleme hinsichtlich Redundanzen und Update-Anomalien.

## Snowflake Schema
Das Snowflake Schema speichert die Fakten (Kennzahlen) in einer zentralen Faktentabelle. Jede Klassifikationsstufe wird dabei in eine eigene Dimensionstabelle gespeichert, die ebenfalls in die dritte Normalform zerlegt gespeichert und mit Fremdschlüsseln referenziert werden. Die Fremdschlüssel bilden dabei auch den Primärschlüssel der Faktentabelle.

Der Name leitet sich von der Form ab. Ausgehend von einer zentralen Faktentabelle werden die Dimensionstabellen referenziert und verzweigen sich durch die dritte Normalform immer weiter.

In [ ]:
%SCHEMA ONLY sf_sales

Der Nachteil wird deutlich, wenn die ursprüngliche Relation wieder erzeugt werden soll. In diesem anschaulichen, aber verhältnismäßig kleinen Beispiel sind dazu bereits sieben Joins nötig.

In [ ]:
SELECT rank, genre, platform, manufacturer, year AS manufacturer_year, city AS headquarter_city, country AS headquarter_country, ceo, decade, sales
FROM sf_sales
    NATURAL JOIN sf_genre
    NATURAL JOIN sf_platforms
        NATURAL JOIN sf_manufacturers
            NATURAL JOIN sf_cities
                NATURAL JOIN sf_countries
            NATURAL JOIN sf_ceos
    NATURAL JOIN sf_decades

## Star Schema
Das Star Schema normalisiert nur die Faktentabelle, belässt die Dimensionstabellen aber denormalisiert (1. Normalform). Der Primärschlüssel der Faktentabelle ergibt sich erneut aus den Fremdschlüsseln, die auf die Dimensionstabellen verweisen.

Der Name leitet sich wieder von der Form ab. Ausgehend von einer zentralen Faktentabelle werden um diese angeordnete, aber nicht weiter verzweigte Dimensionstabellen referenziert.

In [ ]:
%SCHEMA ONLY st_sales

Somit ist nur die Faktentabelle von Redundanzen und Update-Anomalien befreit. Die Anzahl der Joins verringert sich aber entsprechend. Für das gezeigte Beispiel werden nur noch drei Joins benötigt.

In [ ]:
SELECT genre, platform, manufacturer, manufacturer_year, headquarter_city, headquarter_country, ceo, decade, sales, rank
FROM st_sales
    NATURAL JOIN st_genre
    NATURAL JOIN st_platforms
    NATURAL JOIN st_decades

## Zusammenfassung
Das Snowflake- und das Star Schema sind zwei Möglichkeiten, Fakten und Dimensionen in relationalen Datenbanksystemen abzubilden. Die Entscheidung erfolgt unter Abwägung von Redundanzen und Update-Anomalien gegenüber dem Aufwand der Abfrage. Darüber hinaus sind auch Mischformen denkbar, die abhängig von den geplanten Anfragen nur einen Teil der Dimensionstabellen in die dritte Normalform übertragen.

Eine weitere Form, die Erwähnung finden sollte, ist das **Galaxie Schema**. Bei diesem existieren mehrere Faktentabellen, die auch gemeinsame Dimensionstabellen besitzen dürfen.